In [217]:
import fasttext

"""
The training file should be this format:

__label__pp this is my text
__label__psoe this is also my text
"""

'\nThe training file should be this format:\n\n__label__pp this is my text\n__label__psoe this is also my text\n'

In [218]:
import pandas as pd
import numpy as np
import re
import emoji

#Function to get all emojis in a text.
def split_emojis(text):

    emoji_list = []
    data = re.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

#Function to clean the text of a tweet.
def clean_tweet(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"/", "", text)
    text = re.sub(r"RT", "", text)
    text = re.sub("\n+", ' ', text)
    text = re.sub("\r+", ' ', text)
    #text = re.sub(r"@.*", "", text) #This do not improve performance.
    #clean_text = re.sub("RT", "", clean_text)
    #clean_text = re.sub("\n", " ", clean_text)
    #clean_text = re.sub("/", "", clean_text)
    #clean_text = re.sub(" +", " ", clean_text)
    #Separate emojis from words.
    """
    sep_emojis_text = ""
    for c in clean_text:
        if c in emoji.UNICODE_EMOJI:
            sep_emojis_text+=' '+c+' '
        else:
            sep_emojis_text+=c
     #This do not improve performance.
    """
    return text

#Read and clean data for each party.

#Podemos.
up_cleaned_data = pd.DataFrame()
up_df = pd.read_csv ('up.csv').head(4950)
up_cleaned_data["tweet"] = up_df["text"].apply(clean_tweet)
up_cleaned_data["label"] = "__label__up"
print(len(up_df))
#Psoe.
psoe_cleaned_data = pd.DataFrame()
psoe_df = pd.read_csv ('psoe.csv').head(4950)
psoe_cleaned_data["tweet"] = psoe_df["text"].apply(clean_tweet)
psoe_cleaned_data["label"] = "__label__psoe"
print(len(psoe_df))
#CS.
cs_cleaned_data = pd.DataFrame()
cs_df = pd.read_csv ('cs.csv').head(4950)
cs_cleaned_data["tweet"] = cs_df["text"].apply(clean_tweet)
cs_cleaned_data["label"] = "__label__cs"
print(len(cs_df))
#PP.
pp_cleaned_data = pd.DataFrame()
pp_df = pd.read_csv ('pp.csv').head(4950)
pp_cleaned_data["tweet"] = pp_df["text"].apply(clean_tweet)
pp_cleaned_data["label"] = "__label__pp"
print(len(pp_df))
#Vox.
vox_cleaned_data = pd.DataFrame()
vox_df = pd.read_csv ('vox.csv').head(4950)
vox_cleaned_data["tweet"] = vox_df["text"].apply(clean_tweet)
vox_cleaned_data["label"] = "__label__vox"
print(len(vox_df))

#Concat all parties data.
data = pd.concat([up_cleaned_data, psoe_cleaned_data, cs_cleaned_data, pp_cleaned_data, vox_cleaned_data])

4745
4950
4950
4896
4877


In [219]:
#Split data in training and test sets.
import os
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["tweet"], data["label"], test_size=0.15, random_state=42)

training = pd.concat([X_train,y_train], axis=1)
display(training)

test = pd.concat([X_test,y_test], axis=1)
display(test)

#Create training file.
training_file = open("data.train", "w", encoding="utf-8")
for i,r in training.iterrows():
    training_file.write(r["label"] + " " + r["tweet"] + "\n")

#Create test file.
test_file = open("data.test", "w", encoding="utf-8")
for i,r in test.iterrows():
    test_file.write(r["label"] + " " + r["tweet"] + "\n")

,tweet,label
1908,@BorinagaP @manuraortiz @TeoGarciaEgea Pues cr...,__label__psoe
4843,El pacto de Sánchez con ERC y Bildu y el recha...,__label__cs
403,Habla @carmencalvo_ del presupuesto de Memoria...,__label__psoe
1133,@luqasnet @cfcuartero01 @WharfRat_DE Y tan Ile...,__label__vox
1762,"A vuestros hijos podréis contarles que, mientr...",__label__pp
...,...,...
2034,@VOX_Congreso @Ortega_Smith ⛔Última hora Alert...,__label__vox
645,Ojala celebremos juntas la recuperación del #M...,__label__psoe
860,@CAILolocai Felicidades!!!,__label__up
1150,@Bicho1968bicho @populares @pablocasado_ El in...,__label__pp


,tweet,label
2509,"“Si quieres ir rápido camina solo, si quieres ...",__label__cs
1510,@AntonioMaestre Debe ser una buena noticia par...,__label__up
1705,Unos ya comen migajas del Socialcomunismo y a ...,__label__vox
2403,🤐 Así se queda el gobierno socialcomunista cua...,__label__pp
3867,"Sabías que miles de personas con PCR positiva,...",__label__vox
...,...,...
2295,Until when will the EU allow the invasion of t...,__label__vox
2527,@Garcia55982118,__label__vox
1427,@alvarezustarroz Es que es más cómodo... 😜,__label__pp
3922,@dsegoviaatienza,__label__up


In [220]:
import fasttext
# Load input training set.

#By default, fastText sees each training example only five times during training, which is pretty small. The number of times each examples is seen (also known as the number of epochs), can be increased using the -epoch option
model = fasttext.train_supervised(input="data.train", epoch=30)

In [221]:
#To predict one instance.
"""
print( model.predict("Franco deberia volver a vivir") )
print( model.predict("Los españoles son los mejores 💙") )
print( model.predict("Los españoles son los mejores 💚") )
print( model.predict("España es una gran nacion") )
print( model.predict("Pablo Iglesias es el mejor") )
print( model.predict("Pablo Iglesias es un mierdas") )
print( model.predict("Putos progres de mierda") )
print( model.predict("Viva la guardia civil") )
print( model.predict("Igualdad salarial para las mujeres!") )
print( model.predict("La españa moderada") )
print( model.predict("La mujer a trabajar") )
print( model.predict("Viva la fuerza naranja") )
"""

'\nprint( model.predict("Franco deberia volver a vivir") )\nprint( model.predict("Los españoles son los mejores 💙") )\nprint( model.predict("Los españoles son los mejores 💚") )\nprint( model.predict("España es una gran nacion") )\nprint( model.predict("Pablo Iglesias es el mejor") )\nprint( model.predict("Pablo Iglesias es un mierdas") )\nprint( model.predict("Putos progres de mierda") )\nprint( model.predict("Viva la guardia civil") )\nprint( model.predict("Igualdad salarial para las mujeres!") )\nprint( model.predict("La españa moderada") )\nprint( model.predict("La mujer a trabajar") )\nprint( model.predict("Viva la fuerza naranja") )\n'

In [222]:
#To test the test set. (number of samples, precision, recall)
print(model.test("data.test"))

precision = model.test("data.test", k=1)[1]
recall = model.test("data.test", k=1)[2]

F_score = 2*(precision * recall)/(precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("Fscore:", F_score)

(3663, 0.6453726453726454, 0.6453726453726454)
Precision: 0.6453726453726454
Recall: 0.6453726453726454
Fscore: 0.6453726453726454


In [223]:
model.save_model("fastTextModel.model")